### Import Library

In [23]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

### Import Data

In [24]:
X = pd.read_csv('dataset/fitur.csv').values
y = pd.read_csv('dataset/target.csv').values.flatten()

### Split Data

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

### Modeling BBA-SVM

In [26]:
class BinaryBatAlgorithm:
    def __init__(self, n_bats, n_features, max_iter, f_min=0, f_max=2, alpha=0.9, gamma=0.9, pulse_rate=0.5, loudness=1):
        self.n_bats = n_bats
        self.n_features = n_features
        self.max_iter = max_iter
        self.f_min = f_min
        self.f_max = f_max
        self.alpha = alpha
        self.gamma = gamma

        # Initialize positions, velocities, frequencies, pulse rate, and loudness
        self.positions = np.random.randint(2, size=(n_bats, n_features))
        self.velocities = np.zeros((n_bats, n_features))
        self.frequencies = np.zeros(n_bats)
        self.pulse_rate_initial = np.full(n_bats, pulse_rate)
        self.pulse_rate = np.full(n_bats, pulse_rate)
        self.loudness = np.full(n_bats, loudness)

        # Initialize best solution and fitness
        self.best_solution = self.positions[0].copy()
        self.best_fitness = self.fitness(self.best_solution)
        for i in range(self.n_bats):
            fitness = self.fitness(self.positions[i])
            if fitness > self.best_fitness:
                self.best_fitness = fitness
                self.best_solution = self.positions[i].copy()

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def fitness(self, position):
        selected_features = np.where(position == 1)[0]
        # print(selected_features)
        if len(selected_features) == 0:
            return 0  # Penalize empty feature selection
        X_train_selected = X_train[:, selected_features]
        X_test_selected = X_test[:, selected_features]
        model = SVC(kernel='linear', random_state=42)
        model.fit(X_train_selected, y_train)
        y_pred = model.predict(X_test_selected)
        return accuracy_score(y_test, y_pred)

    def optimize(self):
        for t in range(self.max_iter):
            for i in range(self.n_bats):
                # Update frequency
                beta = np.random.rand()
                self.frequencies[i] = self.f_min + (self.f_max - self.f_min) * beta
                # Update velocity
                self.velocities[i] += (self.best_solution - self.positions[i]) * self.frequencies[i]
                # Update position using sigmoid and random threshold
                sigmoid_val = self.sigmoid(self.velocities[i])
                self.positions[i] = (np.random.rand(self.n_features) < sigmoid_val).astype(int)
                
                new_position = self.positions[i].copy()
                new_fitness = self.fitness(new_position)
                # Random walk
                if np.random.rand() < self.pulse_rate[i]:
                    epsilon = np.random.uniform(-1, 1, size=self.n_features)
                    noisy_position = self.best_solution + epsilon * np.mean(self.loudness)
                    new_position = (np.random.rand(self.n_features) < self.sigmoid(noisy_position)).astype(int)
                    new_fitness = self.fitness(new_position)
                    
                if new_fitness > self.best_fitness and np.random.rand() < self.loudness[i]:
                    self.positions[i] = new_position
                    self.best_fitness = new_fitness
                    self.best_solution = new_position.copy()

                    # Update loudness and pulse rate
                    self.loudness[i] *= self.alpha
                    self.pulse_rate[i] = self.pulse_rate_initial[i] * (1 - np.exp(-self.gamma * t))
                # print(self.pulse_rate[i])
            print(f"Iteration {t + 1}, Best Fitness: {self.best_fitness}")

        return self.best_solution, self.best_fitness

In [27]:
# Parameters
n_bats = 50
n_features = X.shape[1]
max_iter = 100

# Run Binary Bat Algorithm
bba = BinaryBatAlgorithm(n_bats=n_bats, n_features=n_features, max_iter=max_iter)
best_solution, best_fitness = bba.optimize()

# Results
print("Best Solution (Selected Features):", np.where(best_solution == 1)[0])
print("Best Fitness (Accuracy):", best_fitness)

Iteration 1, Best Fitness: 0.9473684210526315
Iteration 2, Best Fitness: 0.9473684210526315
Iteration 3, Best Fitness: 0.9473684210526315
Iteration 4, Best Fitness: 0.9473684210526315
Iteration 5, Best Fitness: 0.9473684210526315
Iteration 6, Best Fitness: 0.9473684210526315
Iteration 7, Best Fitness: 0.9473684210526315
Iteration 8, Best Fitness: 0.9473684210526315
Iteration 9, Best Fitness: 0.9473684210526315
Iteration 10, Best Fitness: 0.9473684210526315
Iteration 11, Best Fitness: 0.9473684210526315
Iteration 12, Best Fitness: 0.9473684210526315
Iteration 13, Best Fitness: 0.9473684210526315
Iteration 14, Best Fitness: 0.9473684210526315
Iteration 15, Best Fitness: 0.9473684210526315
Iteration 16, Best Fitness: 0.9473684210526315
Iteration 17, Best Fitness: 0.9473684210526315
Iteration 18, Best Fitness: 0.9473684210526315
Iteration 19, Best Fitness: 0.9473684210526315
Iteration 20, Best Fitness: 0.9473684210526315
Iteration 21, Best Fitness: 0.9473684210526315
Iteration 22, Best Fit